In [1]:
# 패키지 설치
# pip install tensorflow wandb apache-airflow apache-airflow-providers-google

# airflow 설정
# airflow db init
# airflow webserver --port 8080

# 추가 패키지 설치
# Airflow 연동을 위한 google cloud 패키지 필요 여부
# pip install apache-airflow-providers-google
# 설정 확인
# pip show tensorflow wandb apache-airflow

In [1]:
%pip install numpy tensorflow wandb apache-airflow notebook ipykernel


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import wandb
import airflow

print("TensorFlow Version:", tf.__version__)
print("WandB Version:", wandb.__version__)
print("Airflow Version:", airflow.__version__)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

TensorFlow Version: 2.18.0
WandB Version: 0.18.6
Airflow Version: 2.10.3


In [3]:
%pip install wandb


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
import wandb
from wandb.integration.keras import WandbMetricsLogger
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python import PythonOperator
import numpy as np


In [5]:
# Wandb 설정
import wandb

wandb.login()
WANDB_PROJECT = "mlops"
WANDB_ENTITY = "kminimalid"

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kminimalid (kminimalid-google). Use `wandb login --relogin` to force relogin


In [6]:
import tensorflow as tf
import numpy as np
import ssl

def load_and_preprocess_data():
    """MNIST 데이터 로드 및 전처리"""
    # SSL 인증 오류 해결
    ssl._create_default_https_context = ssl._create_unverified_context

    try:
        # MNIST 데이터 로드
        print("MNIST 데이터 로드 중...")
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
        print("MNIST 데이터 로드 완료.")
    except Exception as e:
        print(f"MNIST 데이터 로드 중 오류 발생: {e}")
        return None

    # 데이터 전처리
    print("데이터 전처리 시작...")
    try:
        # 1. 픽셀 값 정규화 (0 ~ 1 사이로 스케일링)
        x_train = x_train.astype(np.float32) / 255.0
        x_test = x_test.astype(np.float32) / 255.0

        # 2. 채널 차원 추가 (28, 28) -> (28, 28, 1)
        x_train = np.expand_dims(x_train, axis=-1)
        x_test = np.expand_dims(x_test, axis=-1)

        # 3. 레이블 데이터를 정수형(int64)으로 변환
        y_train = y_train.astype(np.int64)
        y_test = y_test.astype(np.int64)

        print("데이터 전처리 완료!")
        return (x_train, y_train), (x_test, y_test)

    except Exception as e:
        print(f"데이터 전처리 중 오류 발생: {e}")
        return None

# 함수 호출 예시
data = load_and_preprocess_data()
if data:
    (x_train, y_train), (x_test, y_test) = data
    print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
    print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")
else:
    print("데이터 로드 및 전처리에 실패했습니다.")

MNIST 데이터 로드 중...
MNIST 데이터 로드 완료.
데이터 전처리 시작...
데이터 전처리 완료!
x_train shape: (60000, 28, 28, 1), y_train shape: (60000,)
x_test shape: (10000, 28, 28, 1), y_test shape: (10000,)


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_model(learning_rate=0.001, conv1_filters=32, conv2_filters=64):
    """CNN 모델 생성"""

    try:
        # Sequential 모델 초기화
        model = models.Sequential()

        # 입력 데이터의 형상 확인
        input_shape = (28, 28, 1)
        print(f"입력 데이터 형상: {input_shape}")

        # 첫 번째 Convolutional 층과 MaxPooling 층
        model.add(layers.Conv2D(conv1_filters, (3, 3), activation='relu', input_shape=input_shape))
        model.add(layers.MaxPooling2D((2, 2)))

        # 두 번째 Convolutional 층과 MaxPooling 층
        model.add(layers.Conv2D(conv2_filters, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))

        # Flatten 층 (이미지를 1차원 벡터로 변환)
        model.add(layers.Flatten())

        # Fully Connected (Dense) 층
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(10, activation='softmax'))

        # 모델 컴파일
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        print("모델 생성 및 컴파일 완료!")
        model.summary()  # 모델 구조 출력

        return model

    except Exception as e:
        print(f"모델 생성 중 오류 발생: {e}")
        return None

# 함수 호출 예시
model = create_model()

입력 데이터 형상: (28, 28, 1)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107 UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

모델 생성 및 컴파일 완료!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121,930 (476.29 KB)

 Trainable params: 121,930 (476.29 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
import wandb
import tensorflow as tf
from datetime import datetime
from wandb.integration.keras import WandbMetricsLogger

def train_model(**context):
    """모델 학습 및 W&B 로깅"""

    # W&B 초기화 및 로그인 검증
    try:
        wandb.login()
        print("W&B 로그인 성공.")
    except Exception as e:
        print(f"W&B 로그인 실패: {e}")
        return

    wandb.init(
        project="mnist-classification",
        entity="your_wandb_username",  # W&B 계정 이름 입력
        config={
            "learning_rate": 0.001,
            "conv1_filters": 32,
            "conv2_filters": 64,
            "epochs": 10
        },
        save_code=True
    )

    # 데이터 로드
    print("데이터 로드 중...")
    try:
        (x_train, y_train), (x_test, y_test) = load_and_preprocess_data()
    except Exception as e:
        print(f"데이터 로드 중 오류 발생: {e}")
        wandb.finish()
        return

    # 모델 생성
    print("모델 생성 중...")
    try:
        model = create_model(
            learning_rate=wandb.config.learning_rate,
            conv1_filters=wandb.config.conv1_filters,
            conv2_filters=wandb.config.conv2_filters
        )
    except Exception as e:
        print(f"모델 생성 중 오류 발생: {e}")
        wandb.finish()
        return

    # 모델 학습
    print("모델 학습 시작...")
    try:
        history = model.fit(
            x_train,
            y_train,
            epochs=wandb.config.epochs,
            validation_data=(x_test, y_test),
            callbacks=[WandbMetricsLogger()]
        )
    except Exception as e:
        print(f"모델 학습 중 오류 발생: {e}")
        wandb.finish()
        return

    # 모델 평가
    try:
        test_loss, test_accuracy = model.evaluate(x_test, y_test)
        print(f"Test accuracy: {test_accuracy:.4f}")

        # W&B에 최종 메트릭 기록
        wandb.log({
            "final_test_loss": test_loss,
            "final_test_accuracy": test_accuracy,
            "last_epoch_train_loss": history.history['loss'][-1],
            "last_epoch_train_accuracy": history.history['accuracy'][-1],
            "last_epoch_val_loss": history.history['val_loss'][-1],
            "last_epoch_val_accuracy": history.history['val_accuracy'][-1],
        })

    except Exception as e:
        print(f"모델 평가 중 오류 발생: {e}")
        wandb.finish()
        return

    # 모델 저장
    try:
        model_save_path = f"mnist_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.h5"
        model.save(model_save_path)
        print(f"모델이 저장되었습니다: {model_save_path}")

        # W&B에 모델 아티팩트로 저장
        artifact = wandb.Artifact('mnist-model', type='model')
        artifact.add_file(model_save_path)
        wandb.log_artifact(artifact)

    except Exception as e:
        print(f"모델 저장 중 오류 발생: {e}")

    wandb.finish()
    print("모델 학습 및 로깅이 완료되었습니다.")
    return model_save_path

In [9]:
import wandb

# W&B 프로젝트 및 엔티티 설정
WANDB_PROJECT = "mnist-classification"
WANDB_ENTITY = "your_wandb_username"

def hyperparameter_sweep():
    """W&B를 사용한 하이퍼파라미터 튜닝"""
    
    # W&B 로그인 검증
    try:
        wandb.login()
        print("W&B 로그인 성공.")
    except Exception as e:
        print(f"W&B 로그인 실패: {e}")
        return

    # 스윕 구성 설정
    sweep_config = {
        'method': 'random',  # 랜덤 서치 방법 사용
        'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
        'parameters': {
            'learning_rate': {'values': [0.001, 0.01, 0.0001]},
            'conv1_filters': {'values': [16, 32, 64]},
            'conv2_filters': {'values': [32, 64, 128]},
            'batch_size': {'values': [64, 128, 256]}
        }
    }

    # 스윕 ID 생성
    try:
        sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT, entity=WANDB_ENTITY)
        print(f"스윕 ID 생성 완료: {sweep_id}")
    except Exception as e:
        print(f"스윕 생성 중 오류 발생: {e}")
        return

    # W&B 에이전트 실행 (비동기 문제 해결을 위해 synchronous=True 추가)
    try:
        wandb.agent(sweep_id, function=train_model, count=5, project=WANDB_PROJECT, entity=WANDB_ENTITY, synchronous=True)
        print("하이퍼파라미터 스윕 완료.")
    except Exception as e:
        print(f"W&B 에이전트 실행 중 오류 발생: {e}")

In [10]:
import pendulum
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import wandb

# Airflow 기본 설정
local_tz = pendulum.timezone("Asia/Seoul")

default_args = {
    'owner': 'airflow',
    'start_date': datetime(2024, 1, 1, tzinfo=local_tz),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# DAG 정의
dag = DAG(
    'mnist_training_pipeline',
    default_args=default_args,
    description='MNIST 학습 파이프라인',
    schedule_interval='@daily',
    catchup=False
)

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data():
    import tensorflow as tf
    import numpy as np
    import ssl

    # SSL 인증 오류 해결
    ssl._create_default_https_context = ssl._create_unverified_context

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = x_train.astype(np.float32) / 255.0
    x_test = x_test.astype(np.float32) / 255.0
    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)
    y_train = y_train.astype(np.int64)
    y_test = y_test.astype(np.int64)

    print("데이터 로드 및 전처리 완료!")
    return (x_train, y_train), (x_test, y_test)

# 모델 생성 함수
def create_model(learning_rate=0.001, conv1_filters=32, conv2_filters=64):
    from tensorflow.keras import layers, models
    import tensorflow as tf

    model = models.Sequential([
        layers.Conv2D(conv1_filters, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(conv2_filters, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    print("모델 생성 및 컴파일 완료!")
    return model

# 모델 학습 함수
def train_model():
    import wandb
    from datetime import datetime
    from wandb.integration.keras import WandbMetricsLogger

    wandb.login()
    wandb.init(
        project="mnist-classification",
        entity="your_wandb_username",
        config={
            "learning_rate": 0.001,
            "conv1_filters": 32,
            "conv2_filters": 64,
            "epochs": 10
        }
    )

    (x_train, y_train), (x_test, y_test) = load_and_preprocess_data()
    model = create_model(
        learning_rate=wandb.config.learning_rate,
        conv1_filters=wandb.config.conv1_filters,
        conv2_filters=wandb.config.conv2_filters
    )

    history = model.fit(
        x_train, y_train,
        epochs=wandb.config.epochs,
        validation_data=(x_test, y_test),
        callbacks=[WandbMetricsLogger()]
    )

    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {test_accuracy:.4f}")

    wandb.log({
        "test_loss": test_loss,
        "test_accuracy": test_accuracy
    })

    model_save_path = f"mnist_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.h5"
    model.save(model_save_path)
    print(f"모델이 저장되었습니다: {model_save_path}")

    wandb.finish()

# 하이퍼파라미터 스윕 함수
def hyperparameter_sweep():
    import wandb

    wandb.login()
    sweep_config = {
        'method': 'random',
        'metric': {'name': 'accuracy', 'goal': 'maximize'},
        'parameters': {
            'learning_rate': {'values': [0.001, 0.0001]},
            'conv1_filters': {'values': [32, 64]},
            'conv2_filters': {'values': [64, 128]},
            'epochs': {'values': [5, 10]}
        }
    }

    sweep_id = wandb.sweep(sweep_config, project="mnist-sweep")

    def sweep_train():
        import tensorflow as tf
        from tensorflow.keras import layers, models

        wandb.init()
        (x_train, y_train), (x_test, y_test) = load_and_preprocess_data()

        model = models.Sequential([
            layers.Conv2D(wandb.config.conv1_filters, (3, 3), activation='relu', input_shape=(28, 28, 1)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(wandb.config.conv2_filters, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=wandb.config.learning_rate),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        model.fit(x_train, y_train, epochs=wandb.config.epochs, validation_data=(x_test, y_test))
        wandb.finish()

    wandb.agent(sweep_id, sweep_train, synchronous=True)

# Airflow 태스크 정의
preprocessing_task = PythonOperator(
    task_id='load_and_preprocess_data',
    python_callable=load_and_preprocess_data,
    dag=dag
)

training_task = PythonOperator(
    task_id='train_model',
    python_callable=train_model,
    dag=dag
)

hyperparameter_tuning_task = PythonOperator(
    task_id='hyperparameter_tuning',
    python_callable=hyperparameter_sweep,
    dag=dag
)

# 태스크 의존성 설정
preprocessing_task >> training_task >> hyperparameter_tuning_task

/var/folders/qm/s09l6qvs1nx4fx83gly57t4c0000gn/T/ipykernel_20605/1819432138.py:18 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<Task(PythonOperator): hyperparameter_tuning>

In [15]:
execution_date = datetime(2024, 11, 11, tzinfo=local_tz)
dag.test(execution_date=execution_date)

[2024-11-11T18:03:10.250+0900] {dag.py:4435} INFO - dagrun id: mnist_training_pipeline


NoReferencedTableError: Foreign key associated with column 'dag_run_note.user_id' could not find table 'ab_user' with which to generate a foreign key to target column 'id'